In [1]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
import openai

/home/karysoares/Documents/book-reviews/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

/home/karysoares/Documents/book-reviews/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [3]:
vector_store = Chroma(
    collection_name="books",
    embedding_function=embeddings,
    persist_directory="/home/karysoares/Documents/book-reviews/notebooks/books_vector.db"
)


/home/karysoares/Documents/book-reviews/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [4]:
def generate_answer(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4", 
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

In [5]:
def qa_pipeline(question):
    # Realizando a busca por similaridade
    search_results = vector_store.similarity_search(question, k=5)
    
    # Extraindo o conteúdo dos documentos diretamente da lista retornada
    context = "\n\n".join([doc.page_content for doc in search_results])
    
    prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    answer = generate_answer(prompt)
    return answer

In [6]:
# Executando o pipeline de QA
answer = qa_pipeline("What is a good book on dogs?")
print(answer)

RuntimeError: Cannot open link list file